In [1]:
import pandas as pd

df = pd.read_excel('data/order/浦江非故工单.xlsx')
df['主叫号码'] = df['主叫号码'].astype(str)

In [ ]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from pydantic import BaseModel, Field


class OrderCategory(BaseModel):
    """解析非故障工单。"""

    categoy: Literal['用电咨询', '电价变更', '报装进度查询', '电费矛盾', '其他'] = Field(description='工单问题分类')
    reply_error: str = Field(description='错误信息审核，若无错误则填“无”')
    proess_suggestion: str = Field(description='根据受理内容，*分点*生成对该投诉的处理流程的具体建议。')
    review: str = Field(description='根据受理内容、处理情况，核对工作人员的投诉解决情况')
    predict_socre: int = Field(description='用户对服务的评分', gt=0, le=10)

    def __str__(self):
        return f'工单分类：{self.categoy}\n处理建议：{self.proess_suggestion}\n完成情况：{self.review}\n服务质量评分：{self.predict_socre},'


model = ChatOllama(model='qwen3:8b').with_structured_output(OrderCategory)
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '你是客服助手,处理电力服务系统内的非故障工单。'),
        ('user'),
        (''),
        ('ai'),
        (''),
        ('user', '受理内容：{request}。处理情况：{process_detail}。'),
    ]
)
chain = prompt | model

In [14]:
request, process_detail = df.loc[33, ['受理内容', '处理情况']]  # type: ignore
print(request, process_detail, sep='\n')

【电能表异常】户号为3309947212109的客户反映，电能表异常，不管充电还是不充电电表上有两个灯一直常亮，跟别人家显示的不一样，现申请对电表进行现场检查，请尽快核实处理。
2025年04月02日,金华市浦江县供电公司城关供电所,工作人员王斌对客户户号3309947212109诉求进行处理。经过和处理结果如下：2025年04月02日17时05分首次与客户（15905894550）联系。经核实，来电客户是户号3309947212109的户主。经现场核实，表计接线正确，电能表无异常，客户反映的电表上的两个灯是红外信号灯（详见附件1：现场照片），一直常亮是正常情况，并非是电能表异常。跟别人家显示的不一样是因为别人家的电表是老款电表，客户家的电表是新款电表，故两个电表显示的不一样。现工作人员同客户详细解释，客户表示认可。因国网系统不显示所属区县，故上传2.0系统所属区县截图（详见附件2：2.0营销系统截图）。已于2025年04月03日08时55分将处理情况告知客户。客户意见认可。其他信息：表计异常的表现:无异常。整改措施:无。


In [15]:
ai_msg = chain.invoke({'request': request, 'process_detail': process_detail})
print(ai_msg)


工单分类：电费矛盾
处理建议：1. 客户反映电表异常，但现场检查确认电表接线正确且无异常，客户所见的两个灯为红外信号灯，属正常现象。2. 客户家电表为新款，而他人家为老款，导致显示差异，已向客户解释清楚。3. 建议加强客户电表知识普及，避免类似误解。4. 确保后续处理中详细说明电表功能，减少客户疑虑。
完成情况：处理过程完整，客户已认可，无异常。
服务质量评分：10,


工单分类：其他
处理建议：
1.工作人员应加强业务知识培训，确保在处理客户问题时能够提供准确、高效的解决方案。
2.建议优化网上国网系统，增加手机号码更换时的自动解绑功能，减少客户因手机号码更换带来的不便。
3.加强与客户沟通，确保客户充分了解问题原因及解决方法，提高客户满意度。
完成情况：该处理过程符合规定，工作人员能够及时响应客户需求，解决问题。建议在今后的处理中，进一步优化系统功能，提升客户体验。
服务质量评分：5,